## XBRL US API - Python example  
Based on a notebook created by [Ties de Kok](https://www.tiesdekok.com), this sample Python code queries the XBRL US API (https://xbrl.us/home/use/xbrl-api/)
## Enter your XBRL API credentials 
Run the cell below, then type your XBRL US Web account email, account password, Client ID, and secret as noted, pressing the Enter key on the keyboard after each entry.

In [2]:
print('XBRL US Web account email:')
import os, re, sys, json
import requests
import pandas as pd
import numpy as np
import getpass
email = input()
print('Account password')
password = getpass.getpass()
print('Client ID:')
clientid = getpass.getpass()
print('Client secret:')
secret = getpass.getpass()

········


### Once you've entered your credentials above, Run the cell below to generate an access token. 
  
Request credentials at https://xbrl.us/home/use/xbrl-api-community/#provisioning

In [3]:
body_auth = {'username' : ''.join(email), 
            'client_id': ''.join(clientid), 
            'client_secret' : ''.join(secret), 
            'password' : ''.join(password), 
            'grant_type' : 'password', 
            'platform' : 'ipynb_' + ''.join(email) }
endpoint = 'https://api.xbrl.us'
endpoint_auth = endpoint + '/oauth2/token'
res = requests.post(endpoint_auth, data=body_auth)
auth_json = res.json()
access_token = auth_json['access_token']
print('Your access token is: ' + access_token)
print('This access token expires after 60 minutes. Re-run this cell to generate a new token.')

## Make a query 
After the access token appears above, you can modify the query below, then use the **_Cell >> Run All Below_** menu option from the cell **immediately below this text** to run the entire query for results. 
  
**Once the initial data query is executed**, use *_Run All Below_* menu option at the highest point on this page where you modified the data query to get updated results. 
  
Refer to XBRL API documentation at https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails for other endpoints and parameters to filter and return.

### Define the fields to be returned

In [7]:
fields = ['entity.cik',
         'entity.name.sort(ASC)',
         'dts.id',
         'fact.id',
         'report.filing-date',
         'period.fiscal-year',
         'report.type',
         'concept.local-name',
         'dimensions.count',
         'dimension.local-name.sort(ASC)',
         'member.local-name',
         'fact.value',
         'unit',
         'fact.ultimus-index',
         'dimension.namespace',
         'member.namespace',
         'fact.offset(0)'
         ]

### Filter the query for specific XBRL elements (tags) to research 

In [8]:
XBRL_Elements = ['IntangibleAssetsNetExcludingGoodwill',
                 'FiniteLivedIntangibleAssetsNet',
                 'IndefiniteLivedIntangibleAssetsExcludingGoodwill'
                ]

### Filter companies for the query 

In [9]:
companies_cik = ['0000789019', ## Microsoft (MSFT)
                 '0001018724', ## Amazon (AMZN)
                 '0001652044', ## Alphabet (GOOG)
                ]

### Filter data by years to return

In [10]:
years = ['2019']

### Automatic: assemble API parameters

In [11]:
params = {'concept.local-name': ','.join(XBRL_Elements),
         'fact.has-dimensions': 'TRUE',
         'report.type': '10-K,10-K/A',
         'period.fiscal-period': 'Y',
         'period.fiscal-year': ','.join(years),
         'unit': 'USD',
         'fields': ','.join(fields),
         'entity.cik': ','.join(companies_cik)
         }

In [12]:
search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'
res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
res_json = res.json()
res_df = pd.DataFrame(res_json['data'])

In [13]:
res_df

,entity.cik,entity.name,dts.id,fact.id,report.filing-date,period.fiscal-year,report.type,concept.local-name,dimensions.count,dimension.local-name,member.local-name,fact.value,unit,fact.ultimus-index,dimension.namespace,member.namespace
0,0001652044,Alphabet Inc.,362758,221624796,2020-02-04,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,TradenamesAndOtherMember,353000000,USD,2,http://fasb.org/us-gaap/2019-01-31,http://www.google.com/20191231
1,0001652044,Alphabet Inc.,362758,221624695,2020-02-04,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,CustomerRelationshipsMember,224000000,USD,2,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
2,0001652044,Alphabet Inc.,362758,221625088,2020-02-04,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,PatentsAndDevelopedTechnologyMember,1402000000,USD,2,http://fasb.org/us-gaap/2019-01-31,http://www.google.com/20191231
3,0001018724,"AMAZON.COM, INC.",362382,221546174,2020-01-31,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,TechnologyBasedIntangibleAssetsMember,534000000,USD,1,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
4,0001018724,"AMAZON.COM, INC.",362382,221545329,2020-01-31,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,MarketingRelatedIntangibleAssetsMember,1963000000,USD,1,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
5,0001018724,"AMAZON.COM, INC.",362382,221545455,2020-01-31,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,CustomerRelatedIntangibleAssetsMember,152000000,USD,1,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
6,0001018724,"AMAZON.COM, INC.",362382,221546415,2020-01-31,2019,10-K,IntangibleAssetsNetExcludingGoodwill,1,FiniteLivedIntangibleAssetsByMajorClassAxis,ContractBasedIntangibleAssetsMember,1400000000,USD,1,http://fasb.org/us-gaap/2019-01-31,http://fasb.org/us-gaap/2019-01-31
7,0000789019,MICROSOFT CORPORATION,334516,209185196,2019-08-01,2019,10-K,FiniteLivedIntangibleAssetsNet,1,FiniteLivedIntangibleAssetsByMajorClassAxis,CustomerRelationshipsMember,2924000000,USD,4,http://fasb.org/us-gaap/2018-01-31,http://fasb.org/us-gaap/2018-01-31
8,0000789019,MICROSOFT CORPORATION,334516,209184179,2019-08-01,2019,10-K,FiniteLivedIntangibleAssetsNet,1,FiniteLivedIntangibleAssetsByMajorClassAxis,TechnologyBasedIntangibleAssetsMember,1920000000,USD,4,http://fasb.org/us-gaap/2018-01-31,http://fasb.org/us-gaap/2018-01-31
9,0000789019,MICROSOFT CORPORATION,334516,209184767,2019-08-01,2019,10-K,FiniteLivedIntangibleAssetsNet,1,FiniteLivedIntangibleAssetsByMajorClassAxis,MarketingRelatedIntangibleAssetsMember,2838000000,USD,4,http://fasb.org/us-gaap/2018-01-31,http://fasb.org/us-gaap/2018-01-31
